In [1]:
import numpy as np
import pandas as pd
import torch as torch

In [2]:
dataset_exp=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/DepMap(21Q4)/CCLE/CCLE_expression.csv')
dataset_cn=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/DepMap(21Q4)/CCLE/CCLE_gene_cn.csv')
dataset_crispr=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/DepMap(21Q4)/CRISPR/CRISPR_gene_dependency.csv')
info=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/DepMap(21Q4)/CCLE/sample_info_PrimarySite.csv')

In [8]:
dataset_exp.head(5)
#dataset_cn.head(5)
#dataset_crispr.head(5)
#info.head(5)

,Unnamed: 0,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),...,ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
0,ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,...,1.214125,0.000000,0.111031,0.150560,1.427606,5.781884,0.0,0.000000,0.799087,0.000000
1,ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,...,1.835924,0.000000,0.310340,0.000000,0.807355,4.704319,0.0,0.000000,0.464668,0.070389
2,ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,...,1.823749,0.084064,0.176323,0.042644,1.384050,4.931683,0.0,0.028569,0.263034,0.000000
3,ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,...,0.871844,0.137504,0.263034,2.485427,0.713696,3.858976,0.0,0.000000,0.000000,0.000000
4,ACH-000242,7.465648,0.000000,6.435462,2.414136,3.864929,0.831877,7.198003,5.452530,7.112492,...,2.324811,0.163499,0.163499,0.000000,1.117695,4.990501,0.0,0.000000,0.000000,0.000000


In [9]:
dataset_exp.rename(columns={"Unnamed: 0":"DepMap_ID"}, inplace=True)
dataset_cn.rename(columns={"Unnamed: 0":"DepMap_ID"}, inplace=True)
dataset_crispr.rename(columns={"Unnamed: 0":"DepMap_ID"}, inplace=True)

In [13]:
dataset_crispr.head(5)

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,0.090039,0.014404,0.026854,0.025889,0.072099,0.027321,0.020242,0.292673,0.023156,...,0.036792,0.080288,0.033990,0.007406,0.005243,0.072022,0.010219,0.003482,0.013604,0.325783
1,ACH-000004,0.013243,0.036495,0.092341,0.036627,0.015298,0.082334,0.005894,0.162717,0.007398,...,0.043231,0.228263,0.007126,0.019552,0.016182,0.013051,0.134109,0.006561,0.005777,0.028962
2,ACH-000005,0.049003,0.015630,0.011069,0.005111,0.015789,0.084818,0.005097,0.198673,0.055724,...,0.024989,0.179194,0.052357,0.033687,0.045510,0.076672,0.066256,0.009938,0.021041,0.060251
3,ACH-000007,0.028751,0.064718,0.008623,0.010804,0.023790,0.028661,0.034927,0.172268,0.052594,...,0.203421,0.610637,0.011552,0.002210,0.006742,0.033293,0.136423,0.100970,0.151332,0.279300
4,ACH-000009,0.050127,0.054232,0.013527,0.012925,0.031438,0.078621,0.041898,0.178575,0.033905,...,0.160131,0.454415,0.030754,0.003319,0.066395,0.129228,0.398528,0.037886,0.070438,0.276196


In [16]:
cols_proteingenes=dataset_exp.columns
dataset_cn_pcg=dataset_cn[dataset_cn.columns.intersection(dataset_exp.columns)] #pcg = protein coding genes

In [26]:
dataset_cn_pcg.head(5)

,DepMap_ID,OR4F5 (79501),OR4F29 (729759),OR4F16 (81399),SAMD11 (148398),NOC2L (26155),KLHL17 (339451),PLEKHN1 (84069),PERM1 (84808),HES4 (57801),...,PRY (9081),BPY2 (9083),DAZ1 (1617),DAZ2 (57055),CDY1B (253175),BPY2B (442867),DAZ3 (57054),DAZ4 (57135),BPY2C (442868),CDY1 (9085)
0,ACH-001280,0.914099,0.914099,0.914099,0.914099,0.914099,0.914099,0.914099,0.914099,0.914099,...,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10,7.420145e-10
1,ACH-001521,0.994327,0.994327,0.994327,0.994327,0.994327,0.994327,0.994327,0.994327,0.994327,...,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10,8.678553e-10
2,ACH-000550,0.985890,0.985890,0.985890,0.985890,0.985890,0.985890,0.985890,0.985890,0.985890,...,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01,6.352162e-01
3,ACH-002066,0.011919,1.213783,1.213783,1.213783,1.213783,1.213783,1.213783,1.213783,1.213783,...,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09,6.637833e-09
4,ACH-001959,1.045272,1.045272,1.045272,1.045272,1.045272,1.045272,1.045272,1.045272,1.045272,...,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01,5.626378e-01


In [27]:
len(dataset_cn_pcg.columns) #열 개수 세기

19165

In [28]:
dataset_exp_merge=pd.merge(info, dataset_exp, how='inner', on='DepMap_ID')
dataset_cn_pcg_merge=pd.merge(info, dataset_cn_pcg, how='inner', on='DepMap_ID')
dataset_crispr_merge=pd.merge(info, dataset_crispr, how='inner', on='DepMap_ID')

In [32]:
len(dataset_exp_merge.index) #행 개수 세서 누락된 cell lines 있는지 확인

1389

In [10]:
dataset_exp_merge.head(5)

,DepMap_ID,primary_disease,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),...,ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
0,ACH-000001,Ovarian Cancer,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.0,0.124328,0.0
1,ACH-000002,Leukemia,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.0,0.189034,0.0
2,ACH-000003,Colon/Colorectal Cancer,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.0,0.070389,0.0
3,ACH-000004,Leukemia,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.0,0.782409,0.0
4,ACH-000005,Leukemia,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.0,0.042644,0.0


In [33]:
#Column name을 Symbol로 바꿔주는 코드
#단, 최신 버전으로 업데이트되지 않은 symbol이 많기 때문에 ID로 변환하는 것을 추천
dataset_exp_merge_symbol=dataset_exp_merge.rename(columns={col: col.split('(')[0] for col in dataset_exp_merge.columns})
dataset_cn_pcg_merge_symbol=dataset_cn_pcg_merge.rename(columns={col: col.split('(')[0] for col in dataset_cn_pcg_merge.columns})
dataset_crispr_merge_symbol=dataset_crispr_merge.rename(columns={col: col.split('(')[0] for col in dataset_crispr_merge.columns})

In [34]:
dataset_exp_merge_symbol.head(5)

,DepMap_ID,primary_disease,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,ARHGAP11B,AC004593.2,AC090517.4,AL160269.1,ABCF2-H2BE1,POLR2J3,H2BE1,AL445238.1,GET1-SH3BGR,AC113348.1
0,ACH-000001,Ovarian Cancer,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.0,0.124328,0.0
1,ACH-000002,Leukemia,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.0,0.189034,0.0
2,ACH-000003,Colon/Colorectal Cancer,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.0,0.070389,0.0
3,ACH-000004,Leukemia,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.0,0.782409,0.0
4,ACH-000005,Leukemia,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.0,0.042644,0.0


In [35]:
#Column name을 ID로 바꿔주는 코드
#백업용으로 복사
dataset_exp_merge_ID = dataset_exp_merge
dataset_cn_pcg_merge_ID = dataset_cn_pcg_merge
dataset_crispr_merge_ID = dataset_crispr_merge

In [42]:
#Column name을 ID로 바꿔주는 코드
cols = dataset_crispr_merge_ID.columns.str.extract(r"(^[^()]+$)|\((.+)\)", expand=True)
cols = cols[0].combine_first(cols[1]) #combine_first: cols[0]에 있는 누락값을 cols[1]에서 같은 위치에 있는 값으로 채워넣기
cols

0              DepMap_ID
1        primary_disease
2                      1
3                  29974
4                      2
              ...       
17383             440590
17384              79699
17385               7791
17386              23140
17387              26009
Name: 0, Length: 17388, dtype: object

In [43]:
dataset_crispr_merge_ID.columns = cols

In [44]:
dataset_crispr_merge_ID

,DepMap_ID,primary_disease,1,29974,2,144568,127550,53947,51146,8086,...,55055,11130,7789,158586,79364,440590,79699,7791,23140,26009
0,ACH-000001,Ovarian Cancer,0.090039,0.014404,0.026854,0.025889,0.072099,0.027321,0.020242,0.292673,...,0.036792,0.080288,0.033990,0.007406,0.005243,0.072022,0.010219,0.003482,0.013604,0.325783
1,ACH-000004,Leukemia,0.013243,0.036495,0.092341,0.036627,0.015298,0.082334,0.005894,0.162717,...,0.043231,0.228263,0.007126,0.019552,0.016182,0.013051,0.134109,0.006561,0.005777,0.028962
2,ACH-000005,Leukemia,0.049003,0.015630,0.011069,0.005111,0.015789,0.084818,0.005097,0.198673,...,0.024989,0.179194,0.052357,0.033687,0.045510,0.076672,0.066256,0.009938,0.021041,0.060251
3,ACH-000007,Colon/Colorectal Cancer,0.028751,0.064718,0.008623,0.010804,0.023790,0.028661,0.034927,0.172268,...,0.203421,0.610637,0.011552,0.002210,0.006742,0.033293,0.136423,0.100970,0.151332,0.279300
4,ACH-000009,Colon/Colorectal Cancer,0.050127,0.054232,0.013527,0.012925,0.031438,0.078621,0.041898,0.178575,...,0.160131,0.454415,0.030754,0.003319,0.066395,0.129228,0.398528,0.037886,0.070438,0.276196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,ACH-002508,Skin Cancer,0.089838,0.061359,0.010167,0.031230,0.170170,0.104536,0.052093,0.533288,...,0.038703,0.040901,0.047472,0.003949,0.164107,0.041114,0.082221,0.092833,0.225622,0.013819
1050,ACH-002510,Skin Cancer,0.031648,0.038002,0.023842,0.022829,0.062335,0.016314,0.068709,0.064845,...,0.040279,0.602723,0.039889,0.057356,0.034756,0.091185,0.040483,0.045532,0.102374,0.140863
1051,ACH-002511,Skin Cancer,0.027236,0.021816,0.023611,0.012244,0.073551,0.049142,0.035010,0.200938,...,0.139289,0.189206,0.026823,0.021977,0.028856,0.041945,0.065945,0.074605,0.155427,0.156859
1052,ACH-002874,Skin Cancer,0.148692,0.019789,0.022270,0.009843,0.117717,0.076423,0.092715,0.157309,...,0.718216,0.747870,0.050876,0.051056,0.022082,0.078959,0.086833,0.203711,0.057004,0.335460


In [ ]:
#파일 저장
#dataset_exp_merge_symbol.to_csv("CCLE_expression_Symbol_PrimarySite.csv", header=True, index=False)
#dataset_exp_merge_ID.to_csv("CCLE_expression_ID_PrimarySite.csv", header=True, index=False)

In [201]:
#최신 symbol로 바꿔주는 코드
symbol_entrez=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/DepMap(21Q4)/CCLE/mart_export.csv', dtype=object)
symbol_entrez=symbol_entrez.dropna(how='any',axis=0)
symbol_entrez

,HGNC_symbol,Entrez_Gene_Id
0,A1BG,1
1,A2M,2
2,A2MP1,3
3,NAT1,9
4,NAT2,10
...,...,...
25766,HIKESHIP2,117751740
25769,ZNF45-AS1,118827818
25771,LRRC51,120356739
25772,TOMT,120356740


In [129]:
dataset_recentID=dataset_exp_merge_ID #백업
dataset_recentID 

,DepMap_ID,primary_disease,7105,64102,8813,57147,55732,2268,3075,2519,...,89839,1124,54816,11046,114483834,548644,114483833,647264,106865373,102724657
0,ACH-000001,Ovarian Cancer,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.000000,0.124328,0.000000
1,ACH-000002,Leukemia,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.000000,0.189034,0.000000
2,ACH-000003,Colon/Colorectal Cancer,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.000000,0.070389,0.000000
3,ACH-000004,Leukemia,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.000000,0.782409,0.000000
4,ACH-000005,Leukemia,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.000000,0.042644,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,ACH-002508,Skin Cancer,5.556736,0.000000,6.861955,2.220330,2.843984,0.014355,4.788164,5.529509,...,1.485427,0.000000,0.176323,0.000000,2.056584,3.777157,0.000000,0.000000,0.286881,0.000000
1385,ACH-002509,Skin Cancer,5.548128,0.344828,6.132166,3.249445,4.156235,0.000000,1.981853,5.844737,...,0.948601,0.014355,0.275007,0.000000,0.807355,5.042644,0.000000,0.000000,1.422233,0.000000
1386,ACH-002510,Skin Cancer,4.052242,0.000000,6.295539,1.922198,2.364572,0.000000,1.495695,6.338424,...,0.176323,0.028569,0.042644,0.028569,2.456806,5.454505,0.000000,0.042644,1.454176,0.042644
1387,ACH-002511,Skin Cancer,3.727920,0.000000,7.306335,2.833902,3.942984,0.400538,1.843984,4.697107,...,0.632268,0.028569,0.111031,0.028569,1.570463,4.254745,0.000000,0.000000,0.855990,0.000000


In [204]:
dataset_recentID.columns=dataset_recentID.columns.map(symbol_entrez.set_index('Entrez_Gene_Id')['HGNC_symbol'])

In [205]:
dataset_recentID

,NaN,NaN,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,ARHGAP11B,CHN2,ZNF280D,SLC35D2,ABCF2-H2BE1,POLR2J3,H2BE1,NaN,GET1-SH3BGR,MSANTD5
0,ACH-000001,Ovarian Cancer,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.000000,0.124328,0.000000
1,ACH-000002,Leukemia,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.000000,0.189034,0.000000
2,ACH-000003,Colon/Colorectal Cancer,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.000000,0.070389,0.000000
3,ACH-000004,Leukemia,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.000000,0.782409,0.000000
4,ACH-000005,Leukemia,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.000000,0.042644,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,ACH-002508,Skin Cancer,5.556736,0.000000,6.861955,2.220330,2.843984,0.014355,4.788164,5.529509,...,1.485427,0.000000,0.176323,0.000000,2.056584,3.777157,0.000000,0.000000,0.286881,0.000000
1385,ACH-002509,Skin Cancer,5.548128,0.344828,6.132166,3.249445,4.156235,0.000000,1.981853,5.844737,...,0.948601,0.014355,0.275007,0.000000,0.807355,5.042644,0.000000,0.000000,1.422233,0.000000
1386,ACH-002510,Skin Cancer,4.052242,0.000000,6.295539,1.922198,2.364572,0.000000,1.495695,6.338424,...,0.176323,0.028569,0.042644,0.028569,2.456806,5.454505,0.000000,0.042644,1.454176,0.042644
1387,ACH-002511,Skin Cancer,3.727920,0.000000,7.306335,2.833902,3.942984,0.400538,1.843984,4.697107,...,0.632268,0.028569,0.111031,0.028569,1.570463,4.254745,0.000000,0.000000,0.855990,0.000000


In [207]:
dataset_recentID.columns.values[0]="DepMap_ID"
dataset_recentID.columns.values[1]="primary_disease"

In [209]:
dataset_recentID

,DepMap_ID,primary_disease,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,ARHGAP11B,CHN2,ZNF280D,SLC35D2,ABCF2-H2BE1,POLR2J3,H2BE1,NaN,GET1-SH3BGR,MSANTD5
0,ACH-000001,Ovarian Cancer,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.000000,0.124328,0.000000
1,ACH-000002,Leukemia,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.000000,0.189034,0.000000
2,ACH-000003,Colon/Colorectal Cancer,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.000000,0.070389,0.000000
3,ACH-000004,Leukemia,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.000000,0.782409,0.000000
4,ACH-000005,Leukemia,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.000000,0.042644,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,ACH-002508,Skin Cancer,5.556736,0.000000,6.861955,2.220330,2.843984,0.014355,4.788164,5.529509,...,1.485427,0.000000,0.176323,0.000000,2.056584,3.777157,0.000000,0.000000,0.286881,0.000000
1385,ACH-002509,Skin Cancer,5.548128,0.344828,6.132166,3.249445,4.156235,0.000000,1.981853,5.844737,...,0.948601,0.014355,0.275007,0.000000,0.807355,5.042644,0.000000,0.000000,1.422233,0.000000
1386,ACH-002510,Skin Cancer,4.052242,0.000000,6.295539,1.922198,2.364572,0.000000,1.495695,6.338424,...,0.176323,0.028569,0.042644,0.028569,2.456806,5.454505,0.000000,0.042644,1.454176,0.042644
1387,ACH-002511,Skin Cancer,3.727920,0.000000,7.306335,2.833902,3.942984,0.400538,1.843984,4.697107,...,0.632268,0.028569,0.111031,0.028569,1.570463,4.254745,0.000000,0.000000,0.855990,0.000000


In [212]:
dataset_recentID.columns.isnull().any().any() #단, 최신 Symbol 중에서 누락된 것이 있을 수 있음. 되도록이면 ID 쓸 것.

True